<a href="https://colab.research.google.com/github/nvnhcmus/1M-agents-RL/blob/master/Copie_de_KuramotoSivashinskyRudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates our approach on the Kuramoto Sivashinsky equation using the dataset from Rudy et al
$$
u_t + u_{xxxx} + uu_{x}+u_{xx} = 0
$$

# Kuramoto Sivashinsky Equation
$u_t = -u_{xxxx} - uu_{x}-u_{xx}$

In [5]:
%pylab inline
pylab.rcParams['figure.figsize'] = (9, 6)
import scipy.io as sio
import numpy as np
import pandas as pd
import itertools
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler

Populating the interactive namespace from numpy and matplotlib


# Importing data


In [6]:
# raw=true is important so you download the file rather than the webpage.
!wget https://github.com/snagcliffs/PDE-FIND/blob/master/Datasets/kuramoto_sivishinky.mat?raw=true
# rename the file
!mv kuramoto_sivishinky.mat\?raw\=true kuramoto_sivishinky.mat
data = sio.loadmat('kuramoto_sivishinky.mat')
u = np.real(data['uu'])
x = np.real(data['x'][:,0])
t = np.real(data['tt'][0,:])
(n,m)=u.shape
Xmesh,Tmesh = np.meshgrid(x,t)
Xmesh = Xmesh.T
Tmesh = Tmesh.T
umesh = u
# flatten the data to put into the network
X = np.transpose((Xmesh.flatten(), Tmesh.flatten()))
y = np.real(u.flatten()).reshape(n*m,1)
print(X.shape, y.shape)

--2020-03-30 07:19:00--  https://github.com/snagcliffs/PDE-FIND/blob/master/Datasets/kuramoto_sivishinky.mat?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/snagcliffs/PDE-FIND/raw/master/Datasets/kuramoto_sivishinky.mat [following]
--2020-03-30 07:19:00--  https://github.com/snagcliffs/PDE-FIND/raw/master/Datasets/kuramoto_sivishinky.mat
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/snagcliffs/PDE-FIND/master/Datasets/kuramoto_sivishinky.mat [following]
--2020-03-30 07:19:00--  https://raw.githubusercontent.com/snagcliffs/PDE-FIND/master/Datasets/kuramoto_sivishinky.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.git

In [0]:
# To add noise
noise_level = 0
y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)
# To take a sample of size 20000 for our study
number_of_samples = 10000
np.random.seed(123)
idx = np.random.permutation(y.size)
X_train = X[idx, :][:number_of_samples]
X_train2 = X[idx, :][:number_of_samples] # notes
y_train = y_noisy[idx, :][:number_of_samples]

In [0]:
def calcNorm(vector):
    if (vector.dtype == np.float16):
        vector = vector.astype(np.float32)
    return np.linalg.norm(vector)

# Define a function to calculate derivatives using automatic differentiations 

In [0]:
def cal_deriv(X_train):
    u = y_train.flatten()
    first_col = tf.constant(np.repeat(1.0,len(u)))
    X_traintf = tf.constant(X_train)
    with tf.GradientTape() as g:
        g.watch(X_traintf)
        with tf.GradientTape() as g1:
            g1.watch(X_traintf)
            with tf.GradientTape() as g2:
                g2.watch(X_traintf)
                with tf.GradientTape() as g3:
                    g3.watch(X_traintf)
                    with tf.GradientTape(persistent=True) as g4:
                        g4.watch(X_traintf)
                        res = model.call(X_traintf)
                    deri1 = g4.gradient(res,X_traintf)   # 1st order derivatives
                    
                    # print(deri1.shape)
                    # print(deri1[:,:,:,:,0])
                    # print(deri1[:,:,:,:,1])

                    # u_x, u_t = deri1[:,0],deri1[:,1]
                    u_x, u_t = deri1[:,:,:,:,0],deri1[:,:,:,:,1]
                    del g4  
                    # print(deri1[:,1])

                print('--------------')  
                deri2a = g3.gradient(u_x,X_traintf)
                print(deri2a.shape)
                u_xx = deri2a[:,0]
                # print("------------")
                print(u_xx)
            deri3a = g2.gradient(u_xx,X_traintf)
            u_xxx = deri3a[:,0] 
        deri4a = g1.gradient(u_xxx,X_traintf)
        u_xxxx = deri4a[:,0] 
    deri5a = g.gradient(u_xxxx,X_traintf)
    u_xxxxx  = deri5a[:,0]   
    res = tf.cast(tf.keras.backend.flatten(res),tf.float64)
    Theta = tf.stack((first_col,u_x,u_xx, u_xxx,u_xxxx,u_xxxxx,
                      res, res*u_x, res*u_xx,res*u_xxx,res*u_xxxx,res*u_xxxxx,
                      res**2, res**2*u_x, res**2*u_xx, res**2*u_xxx,res**2*u_xxxx,res**2*u_xxxxx,),axis=1)
    return Theta, u_t, res # res = model.call(X_traintf)
deri_dict = {0:'1', 1: 'u_x', 2: 'u_xx', 3: 'u_xxx',4: 'u_xxxx',5: 'u_xxxxx',
             6: 'u', 7: 'uu_x', 8: 'uu_xx',9: 'uu_xxx',10: 'uu_xxxx',11: 'uu_xxxxx',
             12: 'u^2', 13: 'u^2u_x', 14: 'u^2u_xx', 15:'u^2u_xxx', 16:'u^2u_xxxx', 17:'u^2u_xxxxx'}



# testing
# print(X_train.shape)

# df = cal_deriv(X_train)

In [0]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__(name = 'my_model')
        self.dense_1 = layers.ConvLSTM2D(filters = 64, kernel_size =(1,1),
                                        activation = 'relu',
                                        input_shape = (1,1,1,2))
        self.flat = layers.Flatten()
        self.rep  = layers.RepeatVector(3)
        self.dense_2 = layers.LSTM(50)
        self.dense_3 = layers.LSTM(50)
        self.dense_4 = layers.LSTM(50)
        self.dense_5 = layers.LSTM(50)
        self.dense_6 = layers.LSTM(50)
        self.dense_7 = layers.LSTM(50)
        #self.batn_1 = layers.BatchNormalization()
        self.dense_8 = layers.Dense(1)
    def call(self, inputs):
        # Define your forward pass here
        x = self.dense_1(inputs)
        x = self.flat(x)
        x = self.rep(x)
        x = self.dense_2(x)
        x = self.flat(x)
        x = self.rep(x)
        x = self.dense_3(x)
        x = self.flat(x)
        x = self.rep(x)
        x = self.dense_4(x)
        x = self.flat(x)
        x = self.rep(x)
        x = self.dense_5(x)
        x = self.flat(x)
        x = self.rep(x)
        x = self.dense_6(x)
        x = self.flat(x)
        x = self.rep(x)
        x = self.dense_7(x)
        return self.dense_8(x)

In [12]:
# reshape into subsequences [samples, time steps, rows, cols, channels]
X_train = X_train.reshape((-1,1,1,1,2))
# reshape output into [samples, timesteps, features]
y_train = y_train.reshape((-1,1,1))
model = MyModel()
optimizer = tf.keras.optimizers.Adam(0.002)
model.compile(loss='mse',
                    optimizer=optimizer)#,
                    #metrics=['mae', 'mse'])
history = model.fit(X_train, y_train, epochs=5, batch_size = 1000)#, 
                    #validation_split = 0.2, verbose=0)

y_pred = model.predict(X_train)
norm_val = calcNorm(y_pred-y_train)
norm_val2 = calcNorm(y_train)
print("relative L2 error on train set is {}".format(norm_val/norm_val2))
#print('relative L2 error on train set:',np.linalg.norm(y_pred-y_train,2)/np.linalg.norm(y_train,2))

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Epoch 1/5
10/10 [==============================] - 1s 86ms/step - loss: 1.1082
Epoch 2/5
10/10 [==============================] - 1s 89ms/step - loss: 1.0362
Epoch 3/5
10/10 [==============================] - 1s 146ms/step - loss: 0.9972
Epoch 4/5
10/10 [==============================] - 2s 233ms/step - loss: 0.9769
Epoch 5/5
10/10 [==============================] - 2s 208ms/step - loss: 0.9633
relative L2 error on train set is 106.87023086420035


,loss,epoch
0,1.108198,0
1,1.036230,1
2,0.997233,2
3,0.976879,3
4,0.963279,4


Take a sample of 2000 data points. First, we predict u (denoise) and again, check the performance of the NN before proceeding to the next step.

In [13]:
np.random.seed(321)
idtest = np.random.permutation(y.size)
print(X_train.shape)

X_test = X[idtest, :][:number_of_samples]
y_test = y_noisy[idtest, :][:number_of_samples]

print(X_test.shape)

y_pred = model.predict(X_train)
print('MSE:', np.mean(np.square(y_train-y_pred)))
print('relative L2 error on train set:',np.linalg.norm(y_pred-y_train)/np.linalg.norm(y_train))

X_test = X_test.reshape((-1,1,1,1,2))
print(X_test.shape)

yhat = model.predict(X_test)
print('MSE:', np.mean(np.square(y_test-yhat)))
print('relative L2 error on test set:',np.linalg.norm(yhat-y_test)/np.linalg.norm(y_test))

(10000, 1, 1, 1, 2)
(10000, 2)
MSE: 1.2699906225761433
relative L2 error on train set: 106.87023086420035
(10000, 1, 1, 1, 2)
MSE: 0.9415156699443347
relative L2 error on test set: 0.9224370435806797


The performance is good. We will now calculate the derivatives using automatic differentiations using the function cal_deriv defined above and construct the function library.

**Example (KdV equation)**

True model:

$u_t = -6uu_x - u_{xxx}$

Library of potential terms:

deri_dict = $('1','u','u^2','u^3', 'u^4','u^5','u_{x}','uu_{x}','u^2u_{x}','u^3u_{x}','u^4u_{x}','u^5u_{x}','u_{xx}',
'uu_{xx}','u^2u_{xx}','u^3u_{xx}', 'u^4u_{xx}', 'u^5u_{xx}', 'u_{xxx}', 'uu_{xxx}', 'u^2u_{xxx}', 'u^3u_{xxx}', 'u^4u_{xxx}', 'u^5u_{xxx}', 'u_{xxxx}', 'uu_{xxxx}', 'u^2u_{xxxx}','u^3u_{xxxx}', 'u^4u_{xxxx}', 'u^5u_{xxxx}', 'u_{xxxxx}', 'uu_{xxxxx}', 'u^2u_{xxxxx}','u^3u_{xxxxx}', 'u^4u_{xxxxx}','u^5u_{xxxxx}')$

In [0]:
# reshape into subsequences [samples, time steps, rows, cols, channels]
print(X_train.shape)

df = cal_deriv(X_train)

(10000, 1, 1, 1, 2)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

--------------
(10000, 1, 1, 1, 2)
tf.Tensor(
[[[[ 1.60986721e-03  2.96389102e-04]]]


 [[[-1.13157395e-04  1.96322420e-04]]]


 [[[-1.34400427e-04 -5.13412378e-05]]]


 ...


 [[[-3.35243385e-07 -3.06692498e-04]]]


 [[[-5.63052949e-04 -6.34680619e-04]]]


 [[[ 5.48012741e-03 -1.60495995e-03]]]], shape=(10000, 1, 1, 2), dtype=float64)


# Coefficients Estimation using AIC/BIC

In [24]:
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
import time 
Theta, u_t,res = cal_deriv(X_train)
Theta, u_t = Theta.numpy(), u_t.numpy()

EPSILON = 1e-10
model_bic = LassoLarsIC(criterion='bic')
t1 = time.time()
model_bic.fit(Theta, u_t)
t_bic = time.time() - t1
alpha_bic_ = model_bic.alpha_
print(model_bic.coef_)

model_aic = LassoLarsIC(criterion='aic')
model_aic.fit(Theta, u_t)
alpha_aic_ = model_aic.alpha_
print(model_aic.coef_)

def plot_ic_criterion(model, name, color):
    alpha_ = model.alpha_ + EPSILON
    alphas_ = model.alphas_ + EPSILON
    criterion_ = model.criterion_
    plt.plot(-np.log10(alphas_), criterion_, '--', color=color,
             linewidth=3, label='%s criterion' % name)
    plt.axvline(-np.log10(alpha_), color=color, linewidth=3,
                label='alpha: %s estimate' % name)
    plt.xlabel('-log(alpha)')
    plt.ylabel('criterion')

plt.figure()
plot_ic_criterion(model_aic, 'AIC', 'b')
plot_ic_criterion(model_bic, 'BIC', 'r')
plt.legend()
plt.title('Information-criterion for model selection (training time %.3fs)'
          % t_bic)

InvalidArgumentError: ignored

In [0]:
xi = model_bic.coef_ 
coef_tol = 1e-5           
remaining = np.arange(len(deri_dict))[(np.absolute(xi)>coef_tol)]
num_terms = len(remaining)
non0 = np.array(np.where(np.arange(len(deri_dict))*np.absolute(xi).flatten()>coef_tol)).flatten()
xi = xi[remaining]
l = non0.tolist()
def non0_deri_dict(non0):
  my_dict = {}
  for i in non0:
    my_dict[i]=deri_dict[i]
  return my_dict
w2 = non0_deri_dict(non0)
print(w2,'\n', xi)

{1: 'u_x', 2: 'u_xx', 3: 'u_xxx', 4: 'u_xxxx', 6: 'u', 7: 'uu_x', 8: 'uu_xx', 12: 'u^2', 13: 'u^2u_x'} 
 [-2.53414778e-02  6.17520564e-03  2.11550423e-03  2.86263154e-05
  8.69426946e-03 -7.76886656e-02 -2.89931985e-03 -1.83433182e-03
 -1.14370672e-02]


Drop small coefficients and perform linear regression on remaining coefficients. Those small coefficients are officially dropped if the difference in Loss MSE is smaller than a tol.

# Backward Selection

In [0]:
# To take a sample of size 5000 to validate 
np.random.seed(321)
idx = np.random.permutation(y.size)
X_valid = X[idx, :][:number_of_samples]
y_valid = y_noisy[idx, :][:number_of_samples]

from sklearn.linear_model import LinearRegression

Theta_valid, ut_valid,_ = cal_deriv(X_valid)
Theta_valid, ut_valid = Theta_valid.numpy(), ut_valid.numpy()

def mse_backward_selection(non0, mse_rm_thres = 1e-4):
    for i in non0:
        reg_full = LinearRegression().fit(Theta_valid[:,non0], ut_valid)
        reg_reduced = LinearRegression().fit(
            Theta_valid[:,np.setdiff1d(non0, [i])], ut_valid)
        ful_valid_mse = np.mean(np.square(
            reg_full.predict(Theta_valid[:,non0])-ut_valid))
        redu_valid_mse = np.mean(np.square(reg_reduced.predict(
            Theta_valid[:,np.setdiff1d(non0, [i])])-ut_valid))
        if (redu_valid_mse - ful_valid_mse<mse_rm_thres):
            print(redu_valid_mse - ful_valid_mse)
            non0 = np.setdiff1d(non0, [i]) 
    return non0 
selected_id = mse_backward_selection(non0)
print(selected_id)
coef = LinearRegression().fit(Theta[:,selected_id], u_t).coef_
print(coef)

2.570510229836509e-05
2.8150740483456893e-05
1.2391787549526079e-05
5.277155715951998e-06
4.336932474396926e-07
4.4827587522319745e-05
[1 3 7]
[-0.04331184  0.00170577 -0.07648857]
